In [4]:
all_classes = {"test1", "test2", "test3"}
vehicle_classes = {"bus", "car", "motorcycle", "truck"}
accident_classes = set(all_classes) - vehicle_classes

In [5]:
print("All classes:", all_classes)
print("Vehicle classes:", vehicle_classes)
print("Accident classes:", accident_classes)

All classes: {'test1', 'test2', 'test3'}
Vehicle classes: {'bus', 'car', 'motorcycle', 'truck'}
Accident classes: {'test1', 'test2', 'test3'}


In [ ]:
import cv2
import streamlit as st
import time

def main():
    st.title("Webcam Live Feed")
    run = st.checkbox('Run')
    FRAME_WINDOW = st.image([])
    camera = cv2.VideoCapture(1)
    print(camera.get(cv2.CAP_PROP_FRAME_WIDTH))
    print(camera.get(cv2.CAP_PROP_FRAME_HEIGHT))
    print(f"cv2.CAP_PROP_FPS: {camera.get(cv2.CAP_PROP_FPS)}")

    while run:
        _, frame = camera.read()
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        FRAME_WINDOW.image(frame)

        frame_count = 0
        start_time = time.time()

        while frame_count < 100:
            ret, frame = camera.read()
            if not ret:
                break
            frame_count += 1

        elapsed_time = time.time() - start_time
        actual_fps = frame_count / elapsed_time
        print(f"Actual FPS: {actual_fps}")

    else:
        st.write('Stopped')

In [ ]:
_, frame = camera.read()
frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
frame_count = 0
start_time = t.time()
while frame_count < 100:
    ret, frame = camera.read()
    if not ret:
        break
    frame_count += 1
elapsed_time = t.time() - start_time
actual_fps = frame_count / elapsed_time

# Dapatkan FPS dari webcam FPS = camera.get(cv2.CAP_PROP_FPS) #5
FPS = actual_fps  # Default FPS 
print(f"FPS: {FPS}")
print(f"camera.get(cv2.CAP_PROP_FPS): {camera.get(cv2.CAP_PROP_FPS)}")

In [2]:
import numpy as np

def convert_coordinates(coords, old_res, new_res):
    """
    Mengonversi koordinat dari resolusi asli ke resolusi target.

    Args:
        coords (np.ndarray): Array koordinat dengan bentuk (N, 2).
        old_res (tuple): Resolusi asli (width, height).
        new_res (tuple): Resolusi target (width, height).

    Returns:
        np.ndarray: Koordinat baru pada resolusi target.
    """
    old_width, old_height = old_res
    new_width, new_height = new_res

    scale_x = new_width / old_width
    scale_y = new_height / old_height

    # Skalakan koordinat
    converted_coords = coords * np.array([scale_x, scale_y])
    return converted_coords.astype(int)


# Koordinat asli
source_coords = np.array([
    [619, 394],
    [1032, 423],
    [968, 717],
    [240, 666]
])

# Resolusi asli dan target
old_resolution = (1280, 720)
new_resolution = (1920, 1080)

# Konversi koordinat
converted_coords = convert_coordinates(source_coords, old_resolution, new_resolution)
print("Koordinat baru:", converted_coords)


Koordinat baru: [[ 928  591]
 [1548  634]
 [1452 1075]
 [ 360  999]]


In [1]:
def scale_coordinates(coords, old_res, new_res):
    scale_x = new_res[0] / old_res[0]
    scale_y = new_res[1] / old_res[1]
    
    return [(int(x * scale_x), int(y * scale_y)) for x, y in coords]

# Koordinat asli
coordinates_720p = [(580, 394), (1032, 423), (968, 717), (200, 666)]

# Resolusi
old_resolution = (1280, 720)  # 720p
new_resolution = (640, 360)   # 360p

# Konversi
coordinates_360p = scale_coordinates(coordinates_720p, old_resolution, new_resolution)
print(coordinates_360p)


[(290, 197), (516, 211), (484, 358), (100, 333)]
